# Exploring DeFi Concepts: Token Wrapping, Staking & Arbitrage Signals


This notebook explores simplified decentralized finance (DeFi) mechanisms using Python and Solidity.
Covered topics:

- Basic transactions on Ethereum testnets

- Smart contract deployment and interaction

- Simulated ERC-20 tokens and wrapped tokens

- Chainlink oracle integration



In [ ]:
from web3 import Web3
import solcx
import os
from dotenv import load_dotenv
import json
from eth_typing import ChecksumAddress

### 1. Ethereum transaction simulation

This cell demonstrates a basic local Ethereum transaction using Web3.py and the EthereumTesterProvider. It checks account balances, sends ETH from one account to another, and prints the transaction details.

In [ ]:
# Connect to local Ethereum tester network
w3 = Web3(Web3.EthereumTesterProvider())

assert w3.is_connected(), "Connection failed"

# View initial balances
account_0 = w3.eth.accounts[0]
account_1 = w3.eth.accounts[1]

print(f"Account[0] initial balance: {Web3.from_wei(w3.eth.get_balance(account_0), 'ether')} ETH")
print(f"Account[1] initial balance: {Web3.from_wei(w3.eth.get_balance(account_1), 'ether')} ETH")

# Send ETH from account_0 to account_1
tx = {
    'from': account_0,
    'to': account_1,
    'value': Web3.to_wei(3, 'ether'),
    'gas': 21000,
    'gasPrice': Web3.to_wei(20, 'gwei')
}
tx_hash = w3.eth.send_transaction(tx)

# Get receipt and display results
receipt = w3.eth.get_transaction_receipt(tx_hash)
new_balance = w3.eth.get_balance(account_0)

print(f"Transaction hash: {tx_hash.hex()}")
print(f"New Account[0] balance: {Web3.from_wei(new_balance, 'ether')} ETH")
print(f"Account[1] balance: {Web3.from_wei(w3.eth.get_balance(account_1), 'ether')} ETH")
print(f"Transaction receipt:\n{receipt}")

### 2. DAO Voting Registry (Smart Contract Simulation)

This example simulates a basic on-chain voting registry system. The smart contract keeps track of users’ **voting weights**, which can be used in DAO governance mechanisms or token-based voting logic.

We use Solidity to define a contract that:
- Accepts the registration of voters (with a name and vote weight),
- Allows retrieval of vote weight by name,
- Stores an array of voters for ordered access,
- Computes the total voting weight of all registered participants.

This model is simplified but demonstrates the core mechanisms of decentralized data storage and retrieval using mappings, structs, and arrays.

> This example builds on the structure presented in [Patrick Collins’ Web3.py & Solidity course](https://github.com/smartcontractkit/full-blockchain-solidity-course-py), with modifications to simulate DAO-related logic.


In [ ]:
solcx.install_solc('0.8.0') 
solcx.set_solc_version('0.8.0')

In [ ]:
# Define the Solidity code for the smart contract
    
solidity_code = ''' 
pragma solidity >=0.6.0 <0.9.0;

contract VotingWeightRegistry {
    uint256 totalWeight;

    struct Voter {
        string name;
        uint256 weight;
    }

    Voter[] public voters;
    mapping(string => uint256) public nameToWeight;

    // Add a voter with their voting weight
    function addVoter(string memory _name, uint256 _weight) public {
        voters.push(Voter(_name, _weight));
        nameToWeight[_name] = _weight;
        totalWeight += _weight;
    }

    // Retrieve total weight
    function getTotalWeight() public view returns (uint256) {
        return totalWeight;
    }

    // Lookup by name
    function getWeightByName(string memory _name) public view returns (uint256) {
        return nameToWeight[_name];
    }

    // Retrieve voter by index
    function getVoter(uint index) public view returns (string memory, uint256) {
        Voter memory voter = voters[index];
        return (voter.name, voter.weight);
    }
}
'''

# Compile the Solidity contract
compiled_sol = solcx.compile_source(
    solidity_code,
    output_values=["abi", "bin"]
)

# Extract ABI and bytecode
contract_id, contract_interface = compiled_sol.popitem()
abi = contract_interface['abi']
bytecode = contract_interface['bin']

# Show ABI (for reference or verification)
from pprint import pprint
print("Contract ABI:")
pprint(abi)

In [ ]:
# Set default account for transactions
w3.eth.default_account = w3.eth.accounts[0]

# Create contract in Python using ABI and bytecode
VotingRegistry = w3.eth.contract(abi=abi, bytecode=bytecode)

# Deploy the contract
tx_hash = VotingRegistry.constructor().transact()
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

# Get the deployed contract instance
contract_instance = w3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=abi
)

print("Contract deployed at:", tx_receipt.contractAddress)

# Register a voter with name and vote weight
tx_hash = contract_instance.functions.addVoter("Alice", 5).transact()
w3.eth.wait_for_transaction_receipt(tx_hash)

# Retrieve Alice's vote weight from mapping
vote_weight = contract_instance.functions.getWeightByName("Alice").call()
print("Alice's vote weight:", vote_weight)

# Get the first entry in the voters array
voter_record = contract_instance.functions.voters(0).call()
print("First voter entry:", voter_record)

# Register more voters
contract_instance.functions.addVoter("Bob", 3).transact()
contract_instance.functions.addVoter("Charlie", 7).transact()

# Compute total vote weight
total_weight = sum(contract_instance.functions.nameToWeight(name).call()
                   for name in ["Alice", "Bob", "Charlie"])
print("Total voting weight:", total_weight)

### 3. Chainlink Oracle Integration on Sepolia Testnet

This section demonstrates how to integrate a **Chainlink Oracle** to fetch live ETH/USD price data using the `AggregatorV3Interface`. 

We compile and deploy a Solidity contract to the **Sepolia testnet**, then use Web3.py to interact with it and retrieve the current price from Chainlink's public price feed.

> This code requires a valid **Infura key**, **private key** (with ETH on Sepolia), and a proper Chainlink oracle address for the deployed network. Environment variables are used to avoid hardcoding secrets.

#### Chainlink Oracle Contract (Solidity)

The contract imports Chainlink’s AggregatorV3 interface and exposes a function to retrieve the latest ETH/USD price.


In [ ]:
# a simple oracle usage example
# This contract fetches the latest ETH/USD price from a Chainlink oracle.
# Needs to be deployed on a testnet Sepolia

solidity_code_oracle = '''
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

import "@chainlink/contracts/src/v0.8/interfaces/AggregatorV3Interface.sol";

contract PriceConsumerV3 {
    AggregatorV3Interface internal priceFeed;

    constructor() {
        // ETH/USD on Sepolia
        priceFeed = AggregatorV3Interface(
            0x694AA1769357215DE4FAC081bf1f309aDC325306
        );
    }

    function getLatestPrice() public view returns (int) {
        (, int price,,,) = priceFeed.latestRoundData();
        return price;
    }
}
'''

#Load ENV Variables
load_dotenv()
infura_key = os.getenv("INFURA_KEY")
private_key = os.getenv("PRIVATE_KEY")


#Connect to Sepolia via Infura
w3 = Web3(Web3.HTTPProvider(f"https://sepolia.infura.io/v3/{infura_key}"))
assert w3.is_connected(), "Failed to connect to Sepolia"
account = w3.eth.account.from_key(private_key)
w3.eth.default_account = account.address

# Compile
solcx.install_solc("0.8.0")
compiled = solcx.compile_source(solidity_code_oracle, output_values=["abi", "bin"])
contract_id, interface = compiled.popitem()
abi = interface["abi"]
bytecode = interface["bin"]

# Deploy
contract = w3.eth.contract(abi=abi, bytecode=bytecode)
gas_limit = 300000
gas_price = w3.to_wei("20", "gwei")
tx = contract.constructor().build_transaction({
    "nonce": w3.eth.get_transaction_count(account.address),
    "gas": gas_limit,
    "gasPrice": gas_price
})
signed_tx = w3.eth.account.sign_transaction(tx, private_key)
tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

print(f"Oracle contract deployed at: {tx_receipt.contractAddress}")

# Interact
deployed = w3.eth.contract(address=tx_receipt.contractAddress, abi=abi)
price = deployed.functions.getLatestPrice().call()
print(f"Latest ETH/USD price from Chainlink: ${price / 1e8:.2f}")


### 4. Simulating a Token Wrapping Flow (Mock ERC-20 -> Wrapped Token)

This section demonstrates a simplified simulation of a **token wrapping process** using two Solidity smart contracts and a local Ethereum test environment:

- **`MockToken`**: A basic ERC-20–like token with standard functionality including `transfer`, `approve`, and `transferFrom`. It mints 1M tokens to the deployer on initialization.
- **`WrappedToken`**: A wrapper contract that accepts the `MockToken`, locking the tokens inside itself and issuing internal wrapped balances in return. Users can later withdraw to reclaim their original tokens.

#### Workflow Steps:
1. **Deploy both contracts** locally using `EthereumTesterProvider` (no real ETH or Sepolia needed).
2. **Approve** the `WrappedToken` contract to spend a selected amount of `MockToken` on behalf of the user.
3. **Deposit tokens** into the wrapper, which internally:
   - Calls `transferFrom()` to move `MockToken` from the user to the contract.
   - Increases the internal wrapped balance.
4. **Check wrapped balance** to verify successful deposit.

This simulates the **lock-and-mint pattern** commonly used in wrapped assets (e.g. wETH, wBTC) and cross-chain bridges.

> Note: This is a self-contained local test simulation. It does not interact with real blockchain networks or live token contracts.


In [ ]:
# create a simple mock ERC20 token
mock_token_code = '''
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

contract MockToken {
    mapping(address => uint256) private _balances;
    mapping(address => mapping(address => uint256)) private _allowances;
    uint256 private _totalSupply;
    
    event Transfer(address indexed from, address indexed to, uint256 value);
    event Approval(address indexed owner, address indexed spender, uint256 value);
    
    constructor() {
        _mint(msg.sender, 1000000 * 10**18); // Mint 1M tokens to deployer
    }
    
    function transfer(address to, uint256 amount) public returns (bool) {
        _transfer(msg.sender, to, amount);
        return true;
    }
    
    function approve(address spender, uint256 amount) public returns (bool) {
        _approve(msg.sender, spender, amount);
        return true;
    }
    
    function transferFrom(address from, address to, uint256 amount) public returns (bool) {
        _spendAllowance(from, msg.sender, amount);
        _transfer(from, to, amount);
        return true;
    }
    
    function _transfer(address from, address to, uint256 amount) internal {
        require(from != address(0), "Transfer from zero");
        require(to != address(0), "Transfer to zero");
        require(_balances[from] >= amount, "Insufficient balance");
        
        _balances[from] -= amount;
        _balances[to] += amount;
        emit Transfer(from, to, amount);
    }
    
    function _approve(address owner, address spender, uint256 amount) internal {
        _allowances[owner][spender] = amount;
        emit Approval(owner, spender, amount);
    }
    
    function _spendAllowance(address owner, address spender, uint256 amount) internal {
        require(_allowances[owner][spender] >= amount, "Insufficient allowance");
        _allowances[owner][spender] -= amount;
    }
    
    function _mint(address account, uint256 amount) internal {
        _totalSupply += amount;
        _balances[account] += amount;
        emit Transfer(address(0), account, amount);
    }
    
    // View functions
    function balanceOf(address account) public view returns (uint256) {
        return _balances[account];
    }
    
    function allowance(address owner, address spender) public view returns (uint256) {
        return _allowances[owner][spender];
    }
    
    function totalSupply() public view returns (uint256) {
        return _totalSupply;
    }
}
'''

# create a simplified wrapped token without external dependencies
simplified_wrapped_token = '''
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

contract WrappedToken {
    address public immutable underlying;
    mapping(address => uint256) private _balances;
    uint256 private _totalSupply;
    bool public paused;
    address public owner;
    
    event Deposit(address indexed user, uint256 amount);
    event Withdraw(address indexed user, uint256 amount);
    
    modifier onlyOwner() {
        require(msg.sender == owner, "Not owner");
        _;
    }
    
    constructor(address _underlying) {
        underlying = _underlying;
        owner = msg.sender;
    }
    
    function deposit(uint256 amount) external {
        require(!paused, "Paused");
        require(amount > 0, "Zero amount");
        
        // Transfer tokens using low-level call
        (bool success, bytes memory data) = underlying.call(
            abi.encodeWithSignature(
                "transferFrom(address,address,uint256)", 
                msg.sender, 
                address(this), 
                amount
            )
        );
        require(success && (data.length == 0 || abi.decode(data, (bool))), "Transfer failed");
        
        _balances[msg.sender] += amount;
        _totalSupply += amount;
        emit Deposit(msg.sender, amount);
    }
    
    function withdraw(uint256 amount) external {
        require(!paused, "Paused");
        require(_balances[msg.sender] >= amount, "Insufficient");
        
        _balances[msg.sender] -= amount;
        _totalSupply -= amount;
        
        (bool success, bytes memory data) = underlying.call(
            abi.encodeWithSignature(
                "transfer(address,uint256)",
                msg.sender,
                amount
            )
        );
        require(success && (data.length == 0 || abi.decode(data, (bool))), "Transfer failed");
        
        emit Withdraw(msg.sender, amount);
    }
    
    function balanceOf(address account) external view returns (uint256) {
        return _balances[account];
    }
    
    function totalSupply() external view returns (uint256) {
        return _totalSupply;
    }
    
    function pause() external onlyOwner {
        paused = true;
    }
    
    function unpause() external onlyOwner {
        paused = false;
    }
}
'''

In [ ]:
# Reset the Web3 instance to use local test provider
w3 = Web3(Web3.EthereumTesterProvider())
w3.eth.default_account = w3.eth.accounts[0]

# Compile and deploy both contracts
MockToken = w3.eth.contract(
    abi=solcx.compile_source(mock_token_code)['<stdin>:MockToken']['abi'],
    bytecode=solcx.compile_source(mock_token_code)['<stdin>:MockToken']['bin']
)

# Deploy MockToken
tx_hash = MockToken.constructor().transact()
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
MockToken_instance = w3.eth.contract(address=tx_receipt.contractAddress, abi=MockToken.abi)

# Deploy WrappedToken with MockToken address
WrappedToken = w3.eth.contract(
    abi=solcx.compile_source(simplified_wrapped_token)['<stdin>:WrappedToken']['abi'],
    bytecode=solcx.compile_source(simplified_wrapped_token)['<stdin>:WrappedToken']['bin']
)

tx_hash = WrappedToken.constructor(MockToken_instance.address).transact()
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
WrappedToken_instance = w3.eth.contract(address=tx_receipt.contractAddress, abi=WrappedToken.abi)

# Simulate wrapping process


# Step 1: Check initial balance
initial_balance = MockToken_instance.functions.balanceOf(w3.eth.accounts[0]).call()
print("Initial MockToken balance:", initial_balance)


# Step 2: Approve wrapped token contract to spend
amount_to_approve = 1000 * 10**18
tx_hash = MockToken_instance.functions.approve(WrappedToken_instance.address, amount_to_approve).transact()
w3.eth.wait_for_transaction_receipt(tx_hash)

# Step 3: Deposit into wrapped token contract
deposit_amount = 100 * 10**18
tx_hash = WrappedToken_instance.functions.deposit(deposit_amount).transact()
w3.eth.wait_for_transaction_receipt(tx_hash)

# Step 4: Check wrapped token balance
wrapped_balance = WrappedToken_instance.functions.balanceOf(w3.eth.accounts[0]).call()
print("Wrapped token balance:", wrapped_balance)

### 5. Constant Product Liquidity Pool (Uniswap V2 Simulation)


In [ ]:
from decimal import Decimal, getcontext, InvalidOperation
from typing import Union


TRANSACTION_FEE_FACTOR = Decimal('0.997')

getcontext().prec = 36

In [40]:
def validate_reserves(reserve_in: Decimal, reserve_out: Decimal) -> None:
    """Validate pool reserves"""
    min_reserve = Decimal('1E-18')  # Minimum viable reserve
    max_reserve = Decimal('1E+30')  # Maximum viable reserve
    
    if not (min_reserve <= reserve_in <= max_reserve):
        raise ValueError("Reserve_in out of valid range")
    if not (min_reserve <= reserve_out <= max_reserve):
        raise ValueError("Reserve_out out of valid range")




def get_amount_out(amount_in: Union[int, Decimal], reserve_in: Union[int, Decimal], reserve_out: Union[int, Decimal], max_slippage: Union[int, Decimal] = Decimal('0.005')) -> Decimal:
    """
    Calculate output amount using constant product formula with Decimal precision.
    
    Args:
        amount_in: Input amount
        reserve_in: Input token reserve
        reserve_out: Output token reserve
        max_slippage: Maximum allowed slippage (default 0.5%)
    Returns:
        Decimal: Expected output amount
        
    Raises: 
        ValueError: If inputs are invalid or slippage exceeds limit
    """
    
    # Convert all inputs to Decimal for precision
    try: 
        amount_in_d = Decimal(str(amount_in))
        reserve_in_d = Decimal(str(reserve_in))
        reserve_out_d = Decimal(str(reserve_out))
        max_slippage_d = Decimal(str(max_slippage))
    except InvalidOperation as e:
        print("one of the input parameters is not int or Decimal")
    
    
    # Validate inputs
    if amount_in_d <= 0: 
        raise ValueError(f"Amount must be positive, instead got amount_in = {amount_in_d}")
    if reserve_in_d <= 0 or reserve_out_d <= 0: 
        raise ValueError(f"Reserves must be positive, "
                       f"instead got reserve_in = {reserve_in_d} and reserve_out = {reserve_out_d}")
    
    
    # Validate reserves 
    validate_reserves(reserve_in_d, reserve_out_d)
    
    # constant product rule: x * y = k
    # where x is reserve_in, y is reserve_out, and k is the product of reserves
    
    k = reserve_in_d * reserve_out_d
    
    # Calculate output amount with fee
    new_reserve_in_d = reserve_in_d + amount_in_d*TRANSACTION_FEE_FACTOR
    # k = new_reserve_in_d * (reserve_out_d - amount_out_d)
    amount_out_d = reserve_out_d - (k / new_reserve_in_d)

    # Verufy output amount does not exceed reserves
    if amount_out_d >= reserve_out_d:
        raise ValueError("Insufficient output reserve")
    
    # Calculate and check price impact
    price_before_trade = reserve_out_d/reserve_in_d
    final_price = (reserve_out_d - amount_out_d)/ new_reserve_in_d
    price_impact = abs(Decimal('1') - (final_price / price_before_trade))
    #(amount_out_d / reserve_out_d)
    if price_impact > max_slippage_d: 
        raise ValueError(f"Price impact {price_impact:.2%} exceeds maximum slippage {max_slippage:.2%}")
    
    return amount_out_d

In [41]:
def test_get_amount_out()-> None:
    """Quick tests for get_amount_out()"""    

    # Test setup
    reserve_in = 1000
    reserve_out = 5000
    amount_in = 2

    # Test 1: Basic  swap
    amount_out = get_amount_out(amount_in=amount_in, reserve_in=reserve_in, reserve_out=reserve_out)
    print(f"Test 1 completed: Swap {amount_in} tokens. Amount out is {amount_out:.6f}")
    assert amount_out > 0, "Should return positive amount"
    
    # Test 2: Price impact protection
    try:
        get_amount_out(900, reserve_in, reserve_out, max_slippage=Decimal('0.01'))
        print("Test 2 Failed: Should have raised ValueError")
    except ValueError as e:
        print(f"Test 2 completed: Price impact protection. {str(e)}")
        
    # Test 3: Equal reserves
    amount_out = get_amount_out(amount_in=100, reserve_in=1000, reserve_out=1000, max_slippage=0.5)
    assert abs(amount_out - Decimal('90.661089388')) < Decimal('0.0001'), \
        f"Test 3 failed: amount_out is {amount_out}, expected 90.661089388"
    print(f"Test 3 completed: amount_out is {amount_out:.6f} as expected for amount_in = 100," 
        f"reserve_in and reserve_out 1000")
    
    print("\nAll quick tests completed!")
test_get_amount_out()


def test_get_amount_out_comprehensive():
    """Comprehensive tests for AMM swap calculations"""
    
    test_cases = [
        # Basic cases
        {"in": 100, "r_in": 1000, "r_out": 1000, "m_slip": 0.2,  "expected": Decimal('90.661089388')},
        
        # Edge cases
        {"in": Decimal('0.000001'), "r_in": 1000, "r_out": 1000, "m_slip": 0.3, "expected": Decimal('0.000000997')},
        
        # Large numbers
        {"in": 1000000, "r_in": 10000000, "r_out": 10000000,  "m_slip": 0.9, "expected": Decimal('906610.893880')},
        
        # Imbalanced pools
        {"in": 100, "r_in": 10000, "r_out": 100,  "m_slip": 0.9, "expected": Decimal('0.98715803439')}
    ]
    
    for tc in test_cases:
        result = get_amount_out(tc["in"], tc["r_in"], tc["r_out"], tc["m_slip"])
        assert abs(result - tc["expected"]) < Decimal('0.000001'), \
            f"Failed: got {result}, expected {tc['expected']}"
    print("Comprehensive test completed!")
    
test_get_amount_out_comprehensive()

Test 1 completed: Swap 2 tokens. Amount out is 9.950159
Test 2 completed: Price impact protection. Price impact 72.22% exceeds maximum slippage 1.00%
Test 3 completed: amount_out is 90.661089 as expected for amount_in = 100,reserve_in and reserve_out 1000

All quick tests completed!
Comprehensive test completed!


In [ ]:
def get_amount_in(amount_out: float, reserve_in: float, reserve_out: float) -> float:
    """
    Calculate the amount of input tokens for a given output amount
    using the constant product formula.
    
    :param amount_out: Amount of output tokens
    :param reserve_in: Reserve of input tokens in the pool
    :param reserve_out: Reserve of output tokens in the pool
    :return: Amount of input tokens
    """
    k = reserve_in * reserve_out
    # k = (reserve_in + amount_in*0.997)*(reserve_out - amount_out)
    amount_in =  (k /(reserve_out - amount_out) - reserve_in)/0.997 
    
    return amount_in